# << YouTube comment cralwer >>

## Step1. 크롤러 만들기 

크롤러는 [이곳](https://somjang.tistory.com/45)을 보고 만들었습니다. 아주 좋은 게시글입니다.

## 1. Import pakages

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import pandas as pd

## 2. Define cralwer

In [2]:
#url을 전달하면 해당 페이지에서 댓글을 크롤링해 soup에 받은 후 리턴하는 함수
def comment_crawling(url): 
    chrome_driver = 'C:\selenium\chromedriver' #크롬 드라이버가 설치되어 있는 경로 입력
    driver = wd.Chrome(chrome_driver) #driver에 크롬 드라이버 저장

    url = url #함수의 파라미터로 입력받은 url 전달
    driver.get(url) #get형식으로 호출하기

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string#content-text')))
    #가끔 로딩이 느린 경우가 있어서 댓글이 나올때까지 기다렸다가 크롤링을 수행하도록 설정
    
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight") 

    while True: 
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
        #현재 페이지에 있는 댓글을 가져오고
        time.sleep(3.3)
        #3초마다 스크롤을 아래로 내린다
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        #그리고 3초 뒤에 스크롤을 내렸을 때 또 있다면 그거 가져온다
        
        if new_page_height == last_page_height: #더 이상 스크롤 내릴 게 없다면
            break #while문을 탈출
            
        last_page_height = new_page_height #더 내릴 수 있다면 페이지 height 업데이트


    html_source = driver.page_source #최대한 아래로 내려갔다면 데이터를 받아옴.

    driver.close() #창은 꺼주고

    soup = BeautifulSoup(html_source, 'lxml') #파싱해준다!
    
    return soup #리턴~~~~~~~~!

In [3]:
#soup을 받아 댓글에 대한 정보만 받아 전처리를 통해 저장 후 반환하는 함수
def comment_parsing(soup):
    youtube_comments = soup.select('yt-formatted-string#content-text') #댓글이 저장되어 있는 곳

    str_youtube_comments = [] #최종적으로 정리할 리스트 선언

    for i in range(len(youtube_comments)):
        str_tmp = str(youtube_comments[i].text)

        #전처리를 통해 의미없는 정보를 지워주고
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('                ','')

        str_youtube_comments.append(str_tmp) #지워준 거 최종 리스트에 넣음.
        
    return str_youtube_comments #정제한 데이터 리스트 반환

In [4]:
#정제한 리스트를 pandas의 DataFrame 형태로 바꾸어주는 함수
def to_dataframe(comment_list):
    pd_data = {"Comment" : comment_list} #DataFrmae 형태로 반환받자
    comment = pd.DataFrame(pd_data)
    return comment

In [5]:
#url 리스트를 넘겨 받으면 하나하나씩 언팩킹하여 위의 함수들을 차례대로 실행하는 함수 
def lets_crawling(url_list):
    comment_list = pd.DataFrame({'Comment' : []})

    for i in range(len(url_list)):
        comment = to_dataframe(comment_parsing(comment_crawling(url_list[i])))
        comment_list = pd.concat([comment_list, comment], axis=0, ignore_index=True)

    return comment_list

## 3. Start crawling

#### 제가 가장 많이 보는 롤 유투버인 호진이의 댓글과  
#### 두 번째로 많이 보는 공포게임 유투버 수탉의 댓글을 수집해봅니다

In [ ]:
hojin_url = ['https://www.youtube.com/watch?v=K1nOBTADTBI&t=193s',
             'https://www.youtube.com/watch?v=yh0I8foe28Q']

sutak_url = ['https://www.youtube.com/watch?v=zAJyQ3RKf5Q&list=PLT70pHaK4rK51HVvM0CqNIdtBO0BRsX8t',
            'https://www.youtube.com/watch?v=8qEPM1PhXJ0&list=PLT70pHaK4rK51HVvM0CqNIdtBO0BRsX8t&index=2']

hojins_comment = lets_crawling(hojin_url)
sutaks_comment = lets_crawling(sutak_url)

In [68]:
hojins_comment #유투버 호진이의 두 개 영상 댓글

,Comment
0,"구독, 좋아요 한 번씩 부탁드립니다!재밌는 영상 많이 올라갑니다!"
1,진짜 선수들 멸망전 나오시는데 사람들이 못하니마니 욕안했으면좋겠다
2,프레이가 제대로 하면 진짜 다 개바를거같은데 라고 생각을 하다가 멸망전에서도 웃으면...
3,ㅈㄱ 클템에 서폿 메라면 롤드컵 준우승팀인데?ㅋㅋ
4,비시즌 됐는데 쿠로랑 고릴라도 모이자ㅋㅋㅋㅋ쿠 타이거즈 절대 다시해ㅠㅠ
...,...
600,원래 클템형은 쓰레기랑 tiger인뎈ㅋㅋㅋㄱㄱㄱㄱ
601,Motal?
602,LEGENO
603,1


In [69]:
sutaks_comment #유투버 수탉의 두 개 영상 댓글

,Comment
0,"원래 방송에서 간간이 데바데를 했었는데, 유튜브에 올려달라고 하시는 분들이 많아서 ..."
1,수탉님의 데바데는 레전드가 엄청많아서 이때까지 데바데가 유튜브에 기록되지않은게 너무...
2,와 편집 진짜.. 설명 너무 세세해서 저같은 데린이도 이해하기가 쉬웠어요 편집자님 ...
3,데바데는 무편집본으로 올려도 괜찮을 것 같아욤...!중간중간에 발전기 돌리면서 주위...
4,"데바데가 올라오다니 좀 많이 놀랐당,,(ﾟ∀ﾟ);; 진짜 이러시면... 너무 좋네염ㅎㅎㅎ"
...,...
830,ㅋ
831,★♥꾸어어억 벌써 구독자 30만명을 바라보시는군요 ㅎ 3ㅎ 축하 드립니다!! ㅎㅎ...
832,아니 블포작 하자고 공물 3갠데 그걸 즉처로 죽이고 다니니까 당연히 빡쳐서 랜뽑하짘...
833,Hacker


-------------------------------------------------------

## 4. Save crawling results 

In [ ]:
hojins_comment.to_csv("D:/anaconda/tobigs/assignment/week8/hojins_comment.csv")
sutaks_comment.to_csv("D:/anaconda/tobigs/assignment/week8/sutaks_comment.csv")

In [9]:
#불러오기
hojins_comment = pd.read_csv("D:/anaconda/tobigs/assignment/week8/hojins_comment.csv")
sutaks_comment = pd.read_csv("D:/anaconda/tobigs/assignment/week8/sutaks_comment.csv")

In [10]:
#이제 이걸로 사용!
print(hojins_comment)
print(sutaks_comment)

     Unnamed: 0                                            Comment
0             0               구독, 좋아요 한 번씩 부탁드립니다!재밌는 영상 많이 올라갑니다!
1             1              진짜 선수들 멸망전 나오시는데 사람들이  못하니마니 욕안했으면좋겠다
2             2  프레이가 제대로 하면 진짜 다 개바를거같은데 라고 생각을 하다가 멸망전에서도 웃으면...
3             3                        ㅈㄱ 클템에 서폿 메라면 롤드컵 준우승팀인데?ㅋㅋ
4             4            비시즌 됐는데 쿠로랑 고릴라도 모이자ㅋㅋㅋㅋ쿠 타이거즈 절대 다시해ㅠㅠ
..          ...                                                ...
600         600                        원래 클템형은 쓰레기랑 tiger인뎈ㅋㅋㅋㄱㄱㄱㄱ
601         601                                             Motal?
602         602                                             LEGENO
603         603                                                  1
604         604                정글 롤드컵우승정글러 엠빳다형대려와서 밥타이거즈도 재밋을꺼같은데

[605 rows x 2 columns]
     Unnamed: 0                                            Comment
0             0  원래 방송에서 간간이 데바데를 했었는데, 유튜브에 올려달라고 하시는 분들이 많아서 ...
1             1  수탉님의 데바데는 레전드가 엄청많아서 

## 4. Preprocessing
데이터 프레임의 데이터를 하나의 문자열에 담을 수 있도록 전처리 해줍니다

In [11]:
hojin_list = list(hojins_comment.loc[1:, 'Comment'])  #첫 번째는 댓글이 아닌 영상 소개이므로 두 번째부터 불러옵니다
sutak_list = list(sutaks_comment.loc[1:, 'Comment']) #데이터 프레임 형태로 저장되어 있는 것을 리스트로 반환받습니다
print("Hojin''s Comment : ", len(hojin_list)) 
print("Sutak''s Comment : ", len(sutak_list)) 

Hojin''s Comment :  604
Sutak''s Comment :  834


In [12]:
hojin_str = '\n'.join(hojin_list) #리스트를 리스트 요소마다 ' '로 연결하여 하나의 문자열로 만드는 작업입니다
sutak_str = '\n'.join(sutak_list) #Practical ML때 워드클라우드하다 썼던 코드를 그대로 쓸 수 있었네요 과제 열심히 한게 이렇게 도움이되는 날이 올 줄이야 ㅎ
print("\n < Hojin's 검색어 예시 > \n", hojin_str[:100])
print("\n < Sutak's 검색어 예시 > \n", sutak_str[:100])


 < Hojin's 검색어 예시 > 
 진짜 선수들 멸망전 나오시는데 사람들이  못하니마니 욕안했으면좋겠다
프레이가 제대로 하면 진짜 다 개바를거같은데 라고 생각을 하다가 멸망전에서도 웃으면서 대벌레듀오짓을 하고 있는 

 < Sutak's 검색어 예시 > 
 수탉님의 데바데는 레전드가 엄청많아서 이때까지 데바데가 유튜브에 기록되지않은게 너무 아쉬웟는데, 병아리분들이 녹화하자구떼쓰고떼써서 드디어 유튜브에ㅠㅠ너무좋다ㅠ
와 편집 진짜.. 설
